Crie um sistema capaz de resolver o problema das N-Rainhas através de busca
de melhoria iterativa (hill climbing ou têmpera simulada) para N com os seguintes
valores 10, 20 e 25. Tabele os tempos de processamento para obter uma solução.

In [315]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [316]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
import re
import random
import time

In [317]:
# grid = np.zeros((10, 10), dtype=int)
# print(grid)

In [560]:
# Calcula o score do estado atual com base no número de rainhas que se atacam
# Score = Max - pares de raínhas que se atacam
def score_state(state):
    L = len(state)
    max_atacks = L*(L-1)/2
    state = np.insert(state, 0, 0, axis=0)
    score = 0
    for i in range(1, L+1):
        if i <= L:
            for j in range(i+1, L+1):
                if state[j] == state[i]:
                    score += 1
                elif (i - state[i]) == (j - state[j]):
                    score += 1
                elif state[i] + i == state[j] + j:
                    score += 1
    
    return max_atacks - score

# Calcula a probabilidade de se escolher dados cromossomo com base no seu score
def fitness_function(population):
    total_score = 0
    L = len(population)
    for person in population:
        total_score += score_state(person)
    
    if total_score == 0:
        return np.random.dirichlet(np.ones(L),size=1)[0]
    
    result = np.array([])
    for i in range(L):
        result = np.append(result, score_state(population[i])/total_score)
    
    return result

# Retorna a subpopulação de cromossomos com base na sua probabilidade de ser escolhido
def selection(population, p):
    # Seleção usando o método da roleta. De toda população selecionar uma fração p
    L = len(population)
    
    n = 0
    if int(len(population)*p) % 2 != 0:
        n = int(len(population)*p) - 1
    else:
        n = int(len(population)*p)
            
    selected = np.random.choice(np.arange(0, L, 1), n, p=fitness_function(population), replace=False)
        
    
    result = [population[i] for i in selected]
            
    return result

# Retorna uma nova população de indivíduos após serem cruzados
def cross_over(population):
    selected = np.zeros(len(population)) #1: population[i] já selecionado 0: ainda não selecionado
    new_population = []
    for i in range(len(selected) - 1):
        if selected[i] != 1:
            #Seleciona esse indivíduo
            selected[i] = 1

            #Seleciona outro, à frente no vetor, aleatoriamente
            j = np.random.choice(len(selected))
            while ~(selected[j] == 0 or j <= i):
                j = np.random.choice(len(selected))

            selected[j] = 1
            
            p1 = population[i]
            p2 = population[j]
    
            #Seleciona ao acaso uma posição para o cross
            x = np.random.choice(len(population[0])) + 1

            tmp = p2[:x].copy()
            p2[:x], p1[:x]  = p1[:x], tmp

            new_population.append(p1)
            new_population.append(p2)
        
    
    return new_population

# Adiciona mutação em um gene na fração p de indivíduos da população
def mutation(population, p):
    L = int(len(population)*p)
    #Uma fração de p indivíduos irá sofrer uma mutação em um gene
    for i in range(L):
        #Seleciona ao acaso um indivíduo pra sofrer mutação
        idx = np.random.randint(population.shape[0], size=1)
        cromo = population[idx, :][0]
        
        #Seleciona ao acaso uma posição desse indivíduo
        posic = np.random.choice(len(cromo), 1)
        
        #Seleciona ao acaso um novo gene
        cromo[posic] = np.random.choice(len(cromo), 1) + 1
        
        #Substitui na população
        population[idx] = cromo
        
        
    return population
    

In [597]:
# Enquanto critério de parada não atingido
# stop = False
def search(population, p):
    start = time.perf_counter()
    L = len(population[0])
    maximun = (L*(L-1))/2
    # Calcular aptidão
    steps = 0
    while True:
        steps += 1
        score_aux = 0
        for cromo in population:
            if score_state(cromo) > score_aux:
                score_aux = score_state(cromo)
                cromo_aux = cromo
        
        if score_aux > maximun or len(population) > 200*L:
            finish = time.perf_counter()
            print(f'Finished in {round(finish - start, 3)} second(s)')
            return cromo_aux, steps, score_state(cromo_aux), len(population), population
        
        #Selecionar uma fração p de membros para reprodução
        sub_p = selection(population, p)
        
        #Aplicar cruzamento e adicionar filhos à p
        sub_p = cross_over(selection(population, p))
        for child in sub_p:
            population = np.vstack([population, child])
        
        #Realizar mutação em membros da população
        population = mutation(population, p)
        

In [601]:
population = np.random.randint(1, 11, size=(5, 20))

In [602]:
population

array([[ 8,  8,  9,  9,  3,  7,  2,  9,  3,  7,  5,  6,  9,  4,  2,  8,
        10,  4,  7,  2],
       [ 3,  7,  5,  7,  1,  9,  1,  6,  7,  6,  7,  1,  7,  5,  7,  6,
         8,  3, 10,  1],
       [ 4,  4,  6,  6,  5,  7,  8,  1, 10,  8,  8,  1,  2,  7,  8,  6,
         8,  4,  5,  3],
       [ 1,  3,  6,  4,  9,  2, 10,  9,  1, 10,  1,  7,  9,  2,  3,  8,
         7,  6,  7,  7],
       [ 6,  3,  3,  6,  4,  8,  9,  5,  9,  3,  8,  4,  2,  8,  6,  8,
         8,  1, 10,  4]])

In [603]:
search(population, 0.8)

Finished in 10.921 second(s)


(array([12,  3,  1,  6,  4, 15,  2,  9, 16, 13,  8, 14, 17, 10,  3,  8, 14,
         4, 10,  7]),
 13,
 180.0,
 5015,
 array([[ 4,  3, 15, ...,  3,  3, 17],
        [ 6,  3,  6, ...,  4,  7,  2],
        [ 4,  4,  3, ...,  6, 10,  3],
        ...,
        [ 4,  3,  6, ...,  3, 11,  7],
        [ 4,  3,  3, ...,  4, 12,  2],
        [ 1,  1, 16, ...,  3, 10, 19]]))